In [ ]:
# if hit plotting library issues, try resetting LD path for julia
# can set in ~/.local/share/jupyter/kernels/
haskey(ENV, "LD_LIBRARY_PATH") && @assert ENV["LD_LIBRARY_PATH"] == ""

import Pkg
# use temp or named environment to avoid package clashes across development projects
Pkg.activate(;temp=true)
Pkg.add("Revise")
import Revise

Pkg.develop(path="$(homedir())/workspace/Mycelia")
import Mycelia

pkgs = [
    "ProgressMeter"
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

In [ ]:
PROJECT_BASEDIR = dirname(pwd())
data_dir = joinpath(PROJECT_BASEDIR, "data")
genome_dir = mkpath(joinpath(data_dir, "genomes"))

In [ ]:
modified_fasta_files = sort(filter(x -> occursin(r"\.fna$", x) && occursin(".vcf", x) && occursin("normalized", x), readdir(genome_dir, join=true)), by=x->filesize(x))

In [ ]:
for (i, fasta_file) in enumerate(modified_fasta_files)
    @info "$(fasta_file) = $(i) of $(length(modified_fasta_files))"
    ProgressMeter.@showprogress for coverage in [2^i for i in 0:10]
        @info "simulating $(coverage)x coverage for fasta file $(i) of $(length(modified_fasta_files))"
        Mycelia.simulate_nanopore_reads(fasta = fasta_file, quantity = "$(coverage)x")
    end
end